In [1]:
import numpy as np
import tensorflow as tf
import time
from matplotlib import pyplot as plt

from cleverhans.utils_tf import batch_eval


from utils_config import ModelConfig
from utils_experiment import get_data_dict, train_model
from utils_attacks import get_deltas
from dknn import DkNNModel

In [2]:
plt.rcParams["figure.figsize"] = (15, 10)

In [3]:
mc = ModelConfig(config_file='../configs/config_mnist.yaml',
                 root_dir='../results/')

In [4]:
#train_model(mc)

In [5]:
data_dict = get_data_dict(mc)

# parse data_dict
x_train = data_dict['x_train'] 
labels_train = data_dict['labels_train']
x_test = data_dict['x_test']
y_test = data_dict['y_test']
x_cali = data_dict['x_cali'] 
labels_cali = data_dict['labels_cali']

In [6]:
means = np.array([np.mean(x_train[np.where(labels_train == i)], axis=0) for i in range(10)])
labels = np.arange(10)

In [7]:
N = 4000

In [9]:
# Use Image Parameters.
img_rows, img_cols, nchannels = x_train.shape[1:4]

with mc.get_tensorflow_session() as sess:
    with tf.variable_scope('dknn'):
        model_dir = mc.get_model_dir_name()
        model = mc.load_model(model_dir=model_dir)
        
        # Extract representations for the training and calibration data at each layer of interest to the DkNN.
        layers = ['ReLU1', 'ReLU3', 'ReLU5', 'logits']

        dknn = DkNNModel(
            sess = sess,
            model = model,
            neighbors = mc.nb_neighbors,
            proto_neighbors = mc.nb_proto_neighbors,
            backend = mc.backend,
            nb_classes=mc.nb_classes,
            layers=layers,
            train_data=x_train,
            train_labels=labels_train,
            method='euclidean',
            scope='dknn')
        start = time.time()
        dknn.fit()
        end = time.time()
        print('dknn time', end-start)
        dknn.calibrate(x_cali, labels_cali)
        preds_knn, _, _ = dknn.predict(x_test)

#         points = np.random.choice(x_train.shape[0], N) # to be used for the attack

################# Random points #############################
#         print('starting predicting random points')
#         for i in range(len(points)-1):
#             y, _, _ = dknn.predict(x_train[points[i]:points[i]+1])
#             print('predicted point')
#         print('finished predicting')
#############################################################

################ Attack #####################################
#         def wrapper_dknn(x):
#             return dknn.predict(np.array([x]))
        
#         deltas_k, norms_k = get_deltas(x_train[points],
#                                        labels_train[points],
#                                        means,
#                                        labels,
#                                        wrapper_dknn,
#                                        eps=1e-2)
#############################################################
        
        dknn_geod = DkNNModel(
            sess = sess,
            model = model,
            neighbors = mc.nb_neighbors,
            proto_neighbors = mc.nb_proto_neighbors,
            backend = mc.backend,
            nb_classes=mc.nb_classes,
            layers=layers,
            train_data=x_train,
            train_labels=labels_train,
            method='geodesic',
            neighbors_table_path=mc.get_model_dir_name(),
            scope='dknn')
        start = time.time()
        dknn_geod.fit()
        end = time.time()
        print('gdknn time', end-start)
        dknn_geod.calibrate(x_cali, labels_cali)
        preds_geod, _, _ = dknn_geod.predict(x_test)

##################### Attack ################################
#         def wrapper_geod(x):
#             return dknn_geod.predict(np.array([x]))
        
#         deltas_g, norms_g = get_deltas(x_train[points],
#                                        labels_train[points],
#                                        means,
#                                        labels,
#                                        wrapper_geod,
#                                        eps=1e-2)
#############################################################

############## Plots ########################################
#         fig, axes = plt.subplots(1, 2)
#         ax0, ax1 = axes.flatten()
#         ax0.hist([deltas_k, deltas_g], density=True, histtype='step', cumulative=-1, label=['deltas_dknn', 'deltas_geo'])
#         ax0.set_title('deltas')
#         ax0.legend(prop={'size': 10})
#         ax1.hist([norms_k, norms_g], density=True, histtype='step', cumulative=-1, label=['norms_dknn', 'norms_geo'])
#         ax1.set_title('infinity norms')
#         ax1.legend(prop={'size': 10})
#         plt.show()
#         print('deltas_k', deltas_k)
#         print('deltas_g', deltas_g)
#         print('norms_k', norms_k)
#         print('norms_g', norms_g)  

Reading model from:
 ../results/MNIST/nb_train_1000_lr_0.001_bs_2_1/model.joblib


/home/zekrom/CMU/Fall2019/10715 - Advanced Introduction to Machine Learning/Project/venv/lib/python3.5/site-packages/cleverhans/utils_tf.py:294: UserWarning: batch_eval has moved to cleverhans.evaluation. batch_eval will be removed from utils_tf on or after 2019-03-09.
  warnings.warn("batch_eval has moved to cleverhans.evaluation. "
/home/zekrom/CMU/Fall2019/10715 - Advanced Introduction to Machine Learning/Project/venv/lib/python3.5/site-packages/cleverhans/evaluation.py:376: UserWarning: `args` is deprecated and will be removed on or after 2019-03-09. Pass `batch_size` directly.
  warnings.warn("`args` is deprecated and will be removed on or "


Constructing the NearestNeighbor table
Constructing the NearestNeighbor table
Constructing the NearestNeighbor table
Constructing the NearestNeighbor table
dknn time 9.547266483306885
Starting calibration.
DkNN calibration complete.
Predicting.
Prediction complete.
Constructing the NearestNeighborGeodesic table
Constructing the NearestNeighborGeodesic table
Constructing the NearestNeighborGeodesic table
Constructing the NearestNeighborGeodesic table
gdknn time 0.05647444725036621
Starting calibration.
DkNN calibration complete.
Predicting.
Prediction complete.


In [10]:
print((preds_knn==np.argmax(y_test,axis=1)).mean())
print((preds_geod==np.argmax(y_test,axis=1)).mean())

0.86
0.916
